# $ \text{Т.2} $

$ \text{В таблице приведены данные о содержании иммуноглобулина Ig A} \\ \text{ в сыворотке крови у больных пяти возрастных групп:} $

| № | Содержание Ig A(%)                         |
|:-:|:-------------------------------------------|
| 1 | 83, 85                                     |
| 2 | 84, 85, 85, 86, 86, 87                     |
| 3 | 86, 87, 87, 87, 88, 88, 88, 88, 88, 89, 90 |
| 4 | 89, 90, 90, 91                             |
| 5 | 90, 92                                     |



In [1]:
from rich.console import Console
from rich.table import Table
from rich.panel import Panel

import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.api as sm

from statsmodels.formula.api import ols


console = Console()
table = Table()

$ \text{a) Определить влияние возраста на содержание иммуноглобулина} \\ \text{ в крови с помощью регрессионного анализа.} $

$ \text{b) Провести попарное сравнение средних в рамках регрессионной модели,} \\ \text{ с учётом множественности проверяемых гипотез.} $

In [2]:
data = pd.DataFrame({
    'Age_Group': [1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5],
    'Ig_A': [83, 85, 84, 85, 85, 86, 86, 87, 86, 87, 87, 87, 88, 88, 88, 88, 88, 89, 90, 89, 90, 90, 91, 90, 92]
})

group_data = {
    1: [83, 85],
    2: [84, 85, 85, 86, 86, 87],
    3: [86, 87, 87, 87, 88, 88, 88, 88, 88, 89, 90],
    4: [89, 90, 90, 91],
    5: [90, 92]
}

# Создание индикаторных переменных
base = 3
dummies = pd.get_dummies(data['Age_Group'],
                         prefix='Age', dtype=int).drop(columns=[f'Age_{base}'])

dummies_clear = dummies.copy()

r_squared_data = []
dropped_ages = []

for k in range(1, 5 + 1):
  # Регрессия на остальные переменные
  if (k != base):
    model = sm.OLS(dummies_clear[f'Age_{k}'], sm.add_constant(
      dummies_clear.drop(f'Age_{k}', axis=1))).fit()
    r_squared = model.rsquared
    r_squared_data.append({"Age": f"Age_{k}", "R-squared": f"{r_squared:.3f}"})
    drop_suggestion = False

    if (r_squared > 0.7):
      drop_suggestion = True
      dropped_ages.append(f"Age_{k}")
      dummies_clear = dummies_clear.drop(f'Age_{k}', axis=1)

# Регрессия
X = sm.add_constant(dummies_clear)
model = sm.OLS(data['Ig_A'], X).fit()

In [3]:
table = Table(title="R-squared для возрастных групп",
              show_header=True)

table.add_column("Age Group")
table.add_column("R-squared", style="bold")

for row in r_squared_data:
  table.add_row(row["Age"], row["R-squared"])

console.print(table)

console.print(f"Для всей модели: {model.rsquared:.5f}")

R-squared для возрастных 
          групп          
┏━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Age Group ┃ R-squared ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━┩
│ Age_1     │ 0.080     │
│ Age_2     │ 0.149     │
│ Age_4     │ 0.127     │
│ Age_5     │ 0.080     │
└───────────┴───────────┘

Для всей модели: 0.81061

In [4]:
if dropped_ages:
  console.print(Panel(
    f"Следующие возрастные группы могут быть исключены из-за высоких значений R-square: {', '.join(dropped_ages)}",
    title="Потенциально исключенные возрастные группы",
    border_style="yellow"))
else:
  console.print(Panel("Ни одна возрастная группа не была предложена к исключению.",
                title="Потенциально исключенные возрастные группы", border_style="green"))

╭────────────────────────────────── Потенциально исключенные возрастные группы ───────────────────────────────────╮
│ Ни одна возрастная группа не была предложена к исключению.                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [5]:
params_table = Table(title="Regression Parameters",
                     show_header=True)

params_table.add_column("Parameter")
params_table.add_column("Value", style="bold")

for param, value in model.params.items():
  params_table.add_row(param, f"{value:.3f}")

console.print(params_table)

Regression Parameters 
┏━━━━━━━━━━━┳━━━━━━━━┓
┃ Parameter ┃ Value  ┃
┡━━━━━━━━━━━╇━━━━━━━━┩
│ const     │ 87.818 │
│ Age_1     │ -3.818 │
│ Age_2     │ -2.318 │
│ Age_4     │ 2.182  │
│ Age_5     │ 3.182  │
└───────────┴────────┘

In [6]:
# ANOVA
anova_model = ols('Ig_A ~ C(Age_Group)', data=data).fit()
anova_table = sm.stats.anova_lm(anova_model, typ=2)

anova_table_rich = Table(title="ANOVA Таблица")

labels = ["df", "sum_sq", "F", "PR(>F)"]

anova_table_rich.add_column("Source", style="bold")
for label in labels:
  anova_table_rich.add_column(label, style="bold")

for index, row in anova_table.iterrows():
  anova_table_rich.add_row(
      str(index),
      *[str(row[label]) for label in labels]
  )

console.print("b) Попарное сравнение средних (ANOVA)")
console.print(anova_table_rich, justify="left")

b) Попарное сравнение средних (ANOVA)

                                      ANOVA Таблица                                                                
┏━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓                          
┃ Source       ┃ df   ┃ sum_sq             ┃ F                  ┃ PR(>F)                ┃                          
┡━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩                          
│ C(Age_Group) │ 4.0  │ 99.0236363636357   │ 21.399999999999856 │ 5.407435042473989e-07 │                          
│ Residual     │ 20.0 │ 23.136363636363637 │ nan                │ nan                   │                          
└──────────────┴──────┴────────────────────┴────────────────────┴───────────────────────┘

In [7]:
s_2 = [len(group_data[i + 1]) * (np.std(group_data[i + 1]))**2 /
       (len(group_data[i + 1]) - 1) for i in range(len(group_data))]

pairs = [(i, j) for i in range(1, 6) for j in range(i + 1, 6)]

In [8]:
# --- Проверка на равенство дисперсий ---
console.print("[bold]Проверка на равенство дисперсий[/bold]")
variance_table = Table(show_header=True, header_style="bold")
variance_table.add_column("Группы")
variance_table.add_column("F-статистика", justify="center")
variance_table.add_column("P-значение", justify="center")
variance_table.add_column("Результат", justify="center")

for pair in pairs:
  n = len(group_data[pair[0]])
  m = len(group_data[pair[1]])
  s2x = n / (n - 1) * np.std(group_data[pair[0]]) ** 2
  s2y = m / (m - 1) * np.std(group_data[pair[1]]) ** 2
  delta = s2x / s2y
  p_value = st.f.sf(delta, n - 1, m - 1)

  result_text = "[red]Отвергаем H₀[/red]" if (p_value < 0.05) else "[green]Принимаем H₀[/green]"

  variance_table.add_row(
      f"{pair[0]}, {pair[1]}",
      f"{delta:.4f}",
      f"{p_value:.4f}",
      result_text
  )

console.print(variance_table)

Проверка на равенство дисперсий

┏━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Группы ┃ F-статистика ┃ P-значение ┃  Результат   ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ 1, 2   │    1.8182    │   0.2354   │ Принимаем H₀ │
│ 1, 3   │    1.7188    │   0.2192   │ Принимаем H₀ │
│ 1, 4   │    3.0000    │   0.1817   │ Принимаем H₀ │
│ 1, 5   │    1.0000    │   0.5000   │ Принимаем H₀ │
│ 2, 3   │    0.9453    │   0.4930   │ Принимаем H₀ │
│ 2, 4   │    1.6500    │   0.3609   │ Принимаем H₀ │
│ 2, 5   │    0.5500    │   0.7646   │ Принимаем H₀ │
│ 3, 4   │    1.7455    │   0.3544   │ Принимаем H₀ │
│ 3, 5   │    0.5818    │   0.7808   │ Принимаем H₀ │
│ 4, 5   │    0.3333    │   0.8183   │ Принимаем H₀ │
└────────┴──────────────┴────────────┴──────────────┘

In [9]:
# --- T-тест ---
console.print("[bold]T-тест[/bold]")
ttest_table = Table(show_header=True, header_style="bold")
ttest_table.add_column("Группы")
ttest_table.add_column("Дельта", justify="center")
ttest_table.add_column("P-значение", justify="center")
ttest_table.add_column("Результат", justify="center")

hb_p_values = {}
for pair in pairs:
  delta, p_value = st.ttest_ind(group_data[pair[0]], group_data[pair[1]], equal_var=True)

  if (float(p_value) < 0.05):
    hb_p_values[pair] = p_value
    result_text = "[red]Значимое различие[/red]"
  else:
    result_text = "[green]Незначимое различие[/green]"

  ttest_table.add_row(
      f"{pair[0]}, {pair[1]}",
      f"{delta:.4f}",
      f"{p_value:.4f}",
      result_text
  )

console.print(ttest_table)

T-тест

┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ Группы ┃ Дельта  ┃ P-значение ┃      Результат      ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ 1, 2   │ -1.6432 │   0.1515   │ Незначимое различие │
│ 1, 3   │ -4.4611 │   0.0010   │  Значимое различие  │
│ 1, 4   │ -6.9282 │   0.0023   │  Значимое различие  │
│ 1, 5   │ -4.9497 │   0.0385   │  Значимое различие  │
│ 2, 3   │ -4.2735 │   0.0007   │  Значимое различие  │
│ 2, 4   │ -7.2000 │   0.0001   │  Значимое различие  │
│ 2, 5   │ -6.0249 │   0.0009   │  Значимое различие  │
│ 3, 4   │ -3.6486 │   0.0029   │  Значимое различие  │
│ 3, 5   │ -3.7176 │   0.0034   │  Значимое различие  │
│ 4, 5   │ -1.1547 │   0.3125   │ Незначимое различие │
└────────┴─────────┴────────────┴─────────────────────┘

In [10]:
# --- Поправка Бонферрони ---
console.print("[bold]Поправка Бонферрони[/bold]")
bonferroni_table = Table(show_header=True, header_style="bold")
bonferroni_table.add_column("Группы")
bonferroni_table.add_column("P-значение", justify="center")
bonferroni_table.add_column("Результат", justify="center")


hb_p_values = dict(sorted(hb_p_values.items(), key=lambda item: item[1]))

m = len(hb_p_values)
for pair, p_value in hb_p_values.items():
  if (p_value < 0.05 / m):
    m -= 1
    result_text = "[red]Значимое различие[/red]"
  else:
    result_text = "[green]Незначимое различие[/green]"

  bonferroni_table.add_row(
      f"{pair[0]}, {pair[1]}",
      f"{p_value:.4f}",
      result_text
  )

console.print(bonferroni_table)

Поправка Бонферрони

┏━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Группы ┃ P-значение ┃     Результат     ┃
┡━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ 2, 4   │   0.0001   │ Значимое различие │
│ 2, 3   │   0.0007   │ Значимое различие │
│ 2, 5   │   0.0009   │ Значимое различие │
│ 1, 3   │   0.0010   │ Значимое различие │
│ 1, 4   │   0.0023   │ Значимое различие │
│ 3, 4   │   0.0029   │ Значимое различие │
│ 3, 5   │   0.0034   │ Значимое различие │
│ 1, 5   │   0.0385   │ Значимое различие │
└────────┴────────────┴───────────────────┘